<a href="https://colab.research.google.com/github/NanduRaj/ML/blob/master/CancerPrediction_KerasNueralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing the libraries

In [0]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation

### Utility functions

In [0]:
def print_stats(model, x, y):
    y_pred = model.predict(x)
    y_pred = y_pred.round()
    y_pred = y_pred.astype(int)
    return classification_report(y, y_pred)

In [0]:
def model_generator():
    model = Sequential()
    model.add(Dense(30, activation='sigmoid', input_shape=(30,)))
    model.add(Dense(90, activation='sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(180, activation='sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(90, activation='sigmoid'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.mean_squared_logarithmic_error)
    return model

### Importing the dataset

In [0]:
dataset = pd.read_csv("wisconsin-cancer-dataset.csv")
dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]
dataset.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### Checking the missing values

In [0]:
dataset.isna().sum(axis=0)

id                         0
diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

### Feature Scaling

In [0]:
names = dataset.columns[2:32]
scaler = MinMaxScaler() 
scaled_x = scaler.fit_transform(dataset.iloc[:,2:32]) 
scaled_x = pd.DataFrame(scaled_x, columns=names)
scaled_x

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,0.605518,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,0.141323,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
2,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,0.211247,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
3,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,1.000000,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
4,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,0.186816,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
5,0.258839,0.202570,0.267984,0.141506,0.678613,0.461996,0.369728,0.402038,0.518687,0.551179,...,0.268232,0.312633,0.263908,0.136748,0.712739,0.482784,0.427716,0.598282,0.477035,0.454939
6,0.533343,0.347311,0.523875,0.380276,0.379164,0.274891,0.264058,0.367793,0.370707,0.157119,...,0.531839,0.416844,0.511928,0.349194,0.482269,0.223448,0.302236,0.663918,0.295289,0.187853
7,0.318472,0.376057,0.320710,0.184263,0.598267,0.445126,0.219447,0.297465,0.573737,0.517060,...,0.324795,0.429638,0.299766,0.174941,0.622268,0.330753,0.213898,0.534708,0.321506,0.393939
8,0.284869,0.409537,0.302052,0.159618,0.674099,0.533157,0.435567,0.464861,0.651515,0.504002,...,0.268943,0.498667,0.277852,0.136183,0.654626,0.497531,0.430511,0.707904,0.554504,0.342123
9,0.259312,0.484613,0.277659,0.140997,0.595558,0.675480,0.532568,0.424602,0.489899,0.683867,...,0.254714,0.763859,0.235271,0.129326,0.753682,1.000000,0.882588,0.759450,0.552139,1.000000


### Assigning x and y

In [0]:
x = scaled_x #removing the id column as it cannot be considered a feature
y = dataset.iloc[:,1]
print(x)
print(y)

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
0       0.521037      0.022658        0.545989   0.363733         0.593753   
1       0.643144      0.272574        0.615783   0.501591         0.289880   
2       0.601496      0.390260        0.595743   0.449417         0.514309   
3       0.210090      0.360839        0.233501   0.102906         0.811321   
4       0.629893      0.156578        0.630986   0.489290         0.430351   
..           ...           ...             ...        ...              ...   
564     0.690000      0.428813        0.678668   0.566490         0.526948   
565     0.622320      0.626987        0.604036   0.474019         0.407782   
566     0.455251      0.621238        0.445788   0.303118         0.288165   
567     0.644564      0.663510        0.665538   0.475716         0.588336   
568     0.036869      0.501522        0.028540   0.015907         0.000000   

     compactness_mean  concavity_mean  concave points_mean  sym

### Encoding categorical data values

In [0]:
labelencoder_Y = LabelEncoder()
y = labelencoder_Y.fit_transform(y)
print(y)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0
 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 0 1 0 0 1 

### Splitting into train and test split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 0)
print(x_train)
print(x_test)
print(y_train)
print(y_test)

     radius_mean  texture_mean  perimeter_mean  area_mean  smoothness_mean  \
293     0.230442      0.262090        0.219404   0.122672         0.280672   
332     0.200625      0.343253        0.194527   0.103415         0.476393   
565     0.622320      0.626987        0.604036   0.474019         0.407782   
278     0.312793      0.274941        0.293345   0.181888         0.242394   
489     0.459511      0.354751        0.437496   0.302906         0.201679   
..           ...           ...             ...        ...              ...   
277     0.559847      0.347311        0.532859   0.406575         0.330414   
9       0.259312      0.484613        0.277659   0.140997         0.595558   
359     0.116191      0.291173        0.110773   0.057306         0.435768   
192     0.129632      0.287792        0.117062   0.061336         0.152298   
559     0.214350      0.480893        0.212356   0.110286         0.360928   

     compactness_mean  concavity_mean  concave points_mean  sym

### Training the model

In [0]:
model = model_generator()
model.fit(x_train, y_train, batch_size=30, epochs=2000, verbose=1, validation_data=(x_test, y_test))

W0208 22:15:22.322966 139764387309376 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0208 22:15:22.351296 139764387309376 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0208 22:15:22.354430 139764387309376 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0208 22:15:22.390843 139764387309376 deprecation_wrapper.py:119] From /home/ubuntu/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_defa

Train on 426 samples, validate on 143 samples
Epoch 1/2000
426/426 [==============================] - 1s 2ms/step - loss: 0.1237 - val_loss: 0.1136
Epoch 2/2000
426/426 [==============================] - 0s 84us/step - loss: 0.1194 - val_loss: 0.1121
Epoch 3/2000
426/426 [==============================] - 0s 89us/step - loss: 0.1172 - val_loss: 0.1120
Epoch 4/2000
426/426 [==============================] - 0s 76us/step - loss: 0.1172 - val_loss: 0.1119
Epoch 5/2000
426/426 [==============================] - 0s 109us/step - loss: 0.1169 - val_loss: 0.1121
Epoch 6/2000
426/426 [==============================] - 0s 83us/step - loss: 0.1158 - val_loss: 0.1119
Epoch 7/2000
426/426 [==============================] - 0s 103us/step - loss: 0.1153 - val_loss: 0.1117
Epoch 8/2000
426/426 [==============================] - 0s 95us/step - loss: 0.1158 - val_loss: 0.1118
Epoch 9/2000
426/426 [==============================] - 0s 104us/step - loss: 0.1145 - val_loss: 0.1114
Epoch 10/2000
426/426 [==

426/426 [==============================] - 0s 71us/step - loss: 0.0110 - val_loss: 0.0110
Epoch 80/2000
426/426 [==============================] - 0s 85us/step - loss: 0.0126 - val_loss: 0.0120
Epoch 81/2000
426/426 [==============================] - 0s 77us/step - loss: 0.0100 - val_loss: 0.0112
Epoch 82/2000
426/426 [==============================] - 0s 86us/step - loss: 0.0109 - val_loss: 0.0145
Epoch 83/2000
426/426 [==============================] - 0s 89us/step - loss: 0.0129 - val_loss: 0.0129
Epoch 84/2000
426/426 [==============================] - 0s 87us/step - loss: 0.0095 - val_loss: 0.0107
Epoch 85/2000
426/426 [==============================] - 0s 75us/step - loss: 0.0108 - val_loss: 0.0110
Epoch 86/2000
426/426 [==============================] - 0s 82us/step - loss: 0.0118 - val_loss: 0.0108
Epoch 87/2000
426/426 [==============================] - 0s 75us/step - loss: 0.0107 - val_loss: 0.0124
Epoch 88/2000
426/426 [==============================] - 0s 82us/step - loss: 

426/426 [==============================] - 0s 86us/step - loss: 0.0080 - val_loss: 0.0119
Epoch 158/2000
426/426 [==============================] - 0s 92us/step - loss: 0.0095 - val_loss: 0.0103
Epoch 159/2000
426/426 [==============================] - 0s 86us/step - loss: 0.0084 - val_loss: 0.0102
Epoch 160/2000
426/426 [==============================] - 0s 68us/step - loss: 0.0072 - val_loss: 0.0104
Epoch 161/2000
426/426 [==============================] - 0s 82us/step - loss: 0.0104 - val_loss: 0.0102
Epoch 162/2000
426/426 [==============================] - 0s 73us/step - loss: 0.0102 - val_loss: 0.0120
Epoch 163/2000
426/426 [==============================] - 0s 71us/step - loss: 0.0081 - val_loss: 0.0105
Epoch 164/2000
426/426 [==============================] - 0s 84us/step - loss: 0.0101 - val_loss: 0.0123
Epoch 165/2000
426/426 [==============================] - 0s 74us/step - loss: 0.0080 - val_loss: 0.0106
Epoch 166/2000
426/426 [==============================] - 0s 83us/step

426/426 [==============================] - 0s 85us/step - loss: 0.0082 - val_loss: 0.0103
Epoch 236/2000
426/426 [==============================] - 0s 74us/step - loss: 0.0091 - val_loss: 0.0103
Epoch 237/2000
426/426 [==============================] - 0s 88us/step - loss: 0.0074 - val_loss: 0.0105
Epoch 238/2000
426/426 [==============================] - 0s 92us/step - loss: 0.0094 - val_loss: 0.0102
Epoch 239/2000
426/426 [==============================] - 0s 84us/step - loss: 0.0058 - val_loss: 0.0104
Epoch 240/2000
426/426 [==============================] - 0s 71us/step - loss: 0.0064 - val_loss: 0.0102
Epoch 241/2000
426/426 [==============================] - 0s 85us/step - loss: 0.0070 - val_loss: 0.0128
Epoch 242/2000
426/426 [==============================] - 0s 74us/step - loss: 0.0086 - val_loss: 0.0110
Epoch 243/2000
426/426 [==============================] - 0s 82us/step - loss: 0.0067 - val_loss: 0.0103
Epoch 244/2000
426/426 [==============================] - 0s 74us/step

...

Epoch 1631/2000
426/426 [==============================] - 0s 91us/step - loss: 0.0044 - val_loss: 0.0201
Epoch 1632/2000
426/426 [==============================] - 0s 81us/step - loss: 0.0042 - val_loss: 0.0200
Epoch 1633/2000
426/426 [==============================] - 0s 98us/step - loss: 0.0044 - val_loss: 0.0200
Epoch 1634/2000
426/426 [==============================] - 0s 77us/step - loss: 0.0043 - val_loss: 0.0200
Epoch 1635/2000
426/426 [==============================] - 0s 77us/step - loss: 0.0049 - val_loss: 0.0223
Epoch 1636/2000
426/426 [==============================] - 0s 79us/step - loss: 0.0046 - val_loss: 0.0200
Epoch 1637/2000
426/426 [==============================] - 0s 87us/step - loss: 0.0046 - val_loss: 0.0204
Epoch 1638/2000
426/426 [==============================] - 0s 66us/step - loss: 0.0052 - val_loss: 0.0187
Epoch 1639/2000
426/426 [==============================] - 0s 85us/step - loss: 0.0048 - val_loss: 0.0210
Epoch 1640/2000
426/426 [=====================

426/426 [==============================] - 0s 88us/step - loss: 0.0042 - val_loss: 0.0207
Epoch 1709/2000
426/426 [==============================] - 0s 82us/step - loss: 0.0044 - val_loss: 0.0204
Epoch 1710/2000
426/426 [==============================] - 0s 96us/step - loss: 0.0042 - val_loss: 0.0202
Epoch 1711/2000
426/426 [==============================] - 0s 82us/step - loss: 0.0043 - val_loss: 0.0201
Epoch 1712/2000
426/426 [==============================] - 0s 81us/step - loss: 0.0050 - val_loss: 0.0205
Epoch 1713/2000
426/426 [==============================] - 0s 81us/step - loss: 0.0044 - val_loss: 0.0186
Epoch 1714/2000
426/426 [==============================] - 0s 81us/step - loss: 0.0050 - val_loss: 0.0199
Epoch 1715/2000
426/426 [==============================] - 0s 103us/step - loss: 0.0043 - val_loss: 0.0215
Epoch 1716/2000
426/426 [==============================] - 0s 85us/step - loss: 0.0043 - val_loss: 0.0215
Epoch 1717/2000
426/426 [==============================] - 0s

Epoch 1785/2000
426/426 [==============================] - 0s 124us/step - loss: 0.0045 - val_loss: 0.0190
Epoch 1786/2000
426/426 [==============================] - 0s 97us/step - loss: 0.0044 - val_loss: 0.0186
Epoch 1787/2000
426/426 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0181
Epoch 1788/2000
426/426 [==============================] - 0s 91us/step - loss: 0.0043 - val_loss: 0.0180
Epoch 1789/2000
426/426 [==============================] - 0s 106us/step - loss: 0.0043 - val_loss: 0.0213
Epoch 1790/2000
426/426 [==============================] - 0s 90us/step - loss: 0.0041 - val_loss: 0.0232
Epoch 1791/2000
426/426 [==============================] - 0s 92us/step - loss: 0.0062 - val_loss: 0.0172
Epoch 1792/2000
426/426 [==============================] - 0s 124us/step - loss: 0.0044 - val_loss: 0.0221
Epoch 1793/2000
426/426 [==============================] - 0s 121us/step - loss: 0.0063 - val_loss: 0.0175
Epoch 1794/2000
426/426 [================

426/426 [==============================] - 0s 92us/step - loss: 0.0043 - val_loss: 0.0187
Epoch 1862/2000
426/426 [==============================] - 0s 91us/step - loss: 0.0048 - val_loss: 0.0161
Epoch 1863/2000
426/426 [==============================] - 0s 93us/step - loss: 0.0053 - val_loss: 0.0156
Epoch 1864/2000
426/426 [==============================] - 0s 101us/step - loss: 0.0045 - val_loss: 0.0198
Epoch 1865/2000
426/426 [==============================] - 0s 96us/step - loss: 0.0048 - val_loss: 0.0181
Epoch 1866/2000
426/426 [==============================] - 0s 77us/step - loss: 0.0047 - val_loss: 0.0184
Epoch 1867/2000
426/426 [==============================] - 0s 103us/step - loss: 0.0051 - val_loss: 0.0197
Epoch 1868/2000
426/426 [==============================] - 0s 75us/step - loss: 0.0043 - val_loss: 0.0192
Epoch 1869/2000
426/426 [==============================] - 0s 95us/step - loss: 0.0043 - val_loss: 0.0189
Epoch 1870/2000
426/426 [==============================] - 0

426/426 [==============================] - 0s 94us/step - loss: 0.0042 - val_loss: 0.0180
Epoch 1939/2000
426/426 [==============================] - 0s 86us/step - loss: 0.0043 - val_loss: 0.0179
Epoch 1940/2000
426/426 [==============================] - 0s 110us/step - loss: 0.0043 - val_loss: 0.0180
Epoch 1941/2000
426/426 [==============================] - 0s 104us/step - loss: 0.0043 - val_loss: 0.0188
Epoch 1942/2000
426/426 [==============================] - 0s 87us/step - loss: 0.0042 - val_loss: 0.0196
Epoch 1943/2000
426/426 [==============================] - 0s 95us/step - loss: 0.0042 - val_loss: 0.0198
Epoch 1944/2000
426/426 [==============================] - 0s 97us/step - loss: 0.0044 - val_loss: 0.0193
Epoch 1945/2000
426/426 [==============================] - 0s 80us/step - loss: 0.0045 - val_loss: 0.0196
Epoch 1946/2000
426/426 [==============================] - 0s 97us/step - loss: 0.0042 - val_loss: 0.0199
Epoch 1947/2000
426/426 [==============================] - 0

### Results Evaluation

In [0]:
print('## Train Stats\n', print_stats(model, x_train, y_train))
print('--------------------------------------------------------------')
print('## Test Stats\n', print_stats(model, x_test, y_test))

## Train Stats
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       267
           1       1.00      0.97      0.99       159

    accuracy                           0.99       426
   macro avg       0.99      0.99      0.99       426
weighted avg       0.99      0.99      0.99       426

--------------------------------------------------------------
## Test Stats
               precision    recall  f1-score   support

           0       0.96      0.97      0.96        90
           1       0.94      0.92      0.93        53

    accuracy                           0.95       143
   macro avg       0.95      0.95      0.95       143
weighted avg       0.95      0.95      0.95       143



by 
 [*Nandu Raj, Dexlock*](https://www.linkedin.com/in/nandu-raj-548733147/)

